# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится, она уже сделана.

Необходимо построить модель с максимально большим значением *accuracy*. Чтобы выполнить задачу, нужно довести долю правильных ответов по крайней мере до 0.75.

## Изучение данных

Каждый объект - данные о звонках, минутах разговора, количестве сообщений, потраченных мб трафика и используемый тариф каждого пользователя за месяц. Предобработка уже выполнена. Формат записи тарифа (1 - тариф 'ультра', 0 - 'смарт') подсказывает, что будет задача именно бинарной классификации. 

In [2]:
# импортирование всех нужных библиотек и фуцнкций 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV

# формирование датафрейма 
df = pd.read_csv('dataset_users_behavior.csv', index_col=0)

# просмотр общей информации о датафрейме
df.info()
display(df.head(15))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 150.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


## Разделение данных на выборки

Так как тестовой выборки нет, разобьем исходные данные в соотношении 3:1:1.

In [2]:
# формирование обучающей, валидационной и тестовой выборок
df_train, df_temporary = train_test_split(df, test_size=0.4, random_state=123)
df_valid, df_test = train_test_split(df_temporary, test_size=0.5, random_state=123)

# проверка соотношения размеров выборок
display(len(df_train), len(df_test), len(df_valid))

1928

643

643

## Исследование моделей

Выделение признаков и целевых признаков в тренировочной, валидационной и тестовой выборках.

In [3]:
# признаки и целевой признак в обучающей выборке 
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# признаки и целевой признак в валидационной выборке 
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

# признаки и целевой признак в тестовой выборке 
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Модель 'дерево решений' показала долю правильных ответов на валидационной выборке - **0.82**

Параметры, показывающие лучшие результаты:
* Глубина дерева - 5
* Минимальное число объектов в узле дерева - 2
* Минимальное число объектов в листьях - 3

In [4]:
best_tree_accuracy = 0

# цикл по подбору лучшей совокупности гиперпараметров в модели 'дерево решений' 
for depth in range(1, 11):
    for min_split in range(2, 11):
        for min_leaf in range(1, 11):
            
            model_tree = DecisionTreeClassifier(random_state=123, 
                                                max_depth=depth, min_samples_split=min_split, min_samples_leaf=min_leaf)
            model_tree.fit(features_train, target_train)
            predictions = model_tree.predict(features_valid)
            accuracy_tree = accuracy_score(target_valid, predictions)

            if accuracy_tree > best_tree_accuracy:
                best_tree_model = model_tree
                best_tree_accuracy = accuracy_tree
                best_depth = depth
                best_min_split = min_split
                best_min_leaf = min_leaf

# лучшая точность модели и ее гиперпараметры
display(best_tree_accuracy, best_depth, best_min_split, best_min_leaf)

0.8242612752721618

5

2

3

Модель 'случайный лес' показала долю правильных ответов на валидационной выборке - **0.81**

Параметры, показывающие лучшие результаты:
* Количество деревьев - 21
* Глубина деревьев - 4

In [9]:
best_forest_accuracy = 0

# цикл по подбору лучшей совокупности гиперпараметров в модели 'случайный лес' 
for n in range(1, 41):
    for depth in range(1, 5):
        
        model_forest = RandomForestClassifier(random_state=123, n_estimators=n, max_depth=depth)
        model_forest.fit(features_train, target_train)
        predictions = model_forest.predict(features_valid)
        accuracy_forest = accuracy_score(target_valid, predictions)

        if accuracy_forest > best_forest_accuracy:
            best_forest_model = model_forest
            best_forest_accuracy = accuracy_forest
            best_count_est = n
            best_depth = depth
            
# лучшая точность модели и ее гиперпараметры            
display(best_forest_accuracy, best_count_est, best_depth)

0.8102643856920684

21

4

Модель 'логистическая регрессия' показала долю правильных ответов на валидационной выборке - **0.75**

Алгоритм с лучшими показателями - 'lbfgs'

In [6]:
best_log_regress_accuracy = 0

# цикл по подбору лучшей совокупности гиперпараметров в модели 'логистическая регрессия' 
for algorithm in ['lbfgs', 'sag', 'saga']:

    model_log_regress = LogisticRegression(random_state=123, solver=algorithm)
    model_log_regress.fit(features_train, target_train)
    predictions = model_log_regress.predict(features_valid)
    accuracy_log_regress = accuracy_score(target_valid, predictions)

    if accuracy_log_regress > best_log_regress_accuracy:
        best_log_regress_model = model_log_regress
        best_log_regress_accuracy = accuracy_log_regress
        best_algorithm = algorithm            
            
# лучшая точность модели и ее алгоритм                  
display(best_log_regress_accuracy, best_algorithm)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


0.7589424572317263

'lbfgs'

## Проверка модели на тестовой выборке

Доля правильных ответов на тестовой выборке:
* 'Дерево решений' - 0.77
* 'Случайный лес' - 0.80
* 'Логистическая регрессия' - 0.73

In [7]:
# коэффициенты точности на трех моделях
display(accuracy_score(target_test, best_tree_model.predict(features_test)))
display(accuracy_score(target_test, best_forest_model.predict(features_test)))
display(accuracy_score(target_test, best_log_regress_model.predict(features_test)))

0.7713841368584758

0.8040435458786936

0.7309486780715396

## Проверка модели на адекватность

Точность константной модели, которая работает по принципу присваивания каждому объекту наиболее встречаемый класс, меньше точности выбранной модели 'случайный лес' (69,4% против 80%), что говорит о вменяемости выбранной модели.

In [8]:
# константная модель с наиболее встречаемым классом
model_dummy = DummyClassifier(random_state=123, strategy='most_frequent')
model_dummy.fit(features_train, target_train)

# точность константной и выбранной моделей
display(accuracy_score(target_test, model_dummy.predict(features_test)))
display(accuracy_score(target_test, best_forest_model.predict(features_test)))

0.6936236391912908

0.8040435458786936

## Вывод

* Поделили данные на три выборки (обучающая, валидационная, тестовая) в соотношении 3:1:1; в каждой выборке выделили признаки и целевой признак; в циклах подобрали совокупности гиперпараметров для наилучшей точности по трем моделям: дерево решений, случайный лес, логистическая регрессия; сравнили модели и нашли самую подходящую - случайный лес.
* Модель 'дерево решений' на валидационной выборке показала значение 0.81 против 0.77 на тестовой, что говорит о переобучении модели; модель 'логистическая регрессия' показывает не высокие результаты и на валидационной выборке (0.75) и на тестовой (0.73), возможно дело в недообучени.
* Наилучшие результаты показывает модель 'случайный лес', на валидационной выборке - 0.81 и на тестовой - 0.80. Единственный минус модели в скорости работы по сравнению с 'деревом решений' и 'логистической регрессией'.
* Из трех представленных моделей для подборки тарифа наиболее подходит (показывает наивысшую точность) 'случайный лес'.